In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
#ignore comment

In [3]:
import pandas as pd
import numpy as np

import pyarrow.parquet as pq

import time
import datetime as dt


from pathlib import Path

#### Adjust pandas Outcput

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#### LOAD DATA:
1. <b>CANARY(TREATED)
2. <b>BIOMETRICS(OUTCOME)
3. <b>PMPM(ELIGIBLE)

In [5]:
%%time
canary_p = '../../data/CANARY/CANARY/PERSON_SUMMARY'
biometrics_p = '../../data/BIOMETRICS'
pmpm_p = '../../temp/PMPM.p'

canary = pq.read_table(canary_p).to_pandas()
biometrics = pq.read_table(biometrics_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)


CPU times: user 27.4 s, sys: 9.17 s, total: 36.5 s
Wall time: 19.4 s


In [20]:
pmpm.head(2)

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  \
0  2561296               38        158        0             0                  0       0          0                    0             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          0                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0               0   
1  2561296               39          0        0             0                  0       0          0                    0             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          0                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0               0   

   CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR     MYR  
0            0                   0        0                 0                       0               0                0          0           0                   0                 0         1  201807  
1            0                   0        0                 0                       0               0                0          0           0                   0                 0         1  201808

#### Data Processing and Filtering
1. For Biometric data
2. For PMPM Data
3. For Canary data

#### 1) Biometric data 
1. Pick Category of data
1. Standarize Dates: MYR, Datetime Date format
2. Average Values: In the same Month Year

#### An inspection of categories registered in biometrics data

In [8]:
%%time
biometrics.groupby(by = 'BIOMETRIC_CATEGORY').apply(len)

CPU times: user 19.7 s, sys: 5.21 s, total: 25 s
Wall time: 24.9 s


BIOMETRIC_CATEGORY
A1C                             790853
BMI                           10034183
BNP                              42552
Blood Pressure (Diastolic)     4902004
Blood Pressure (Systolic)      5238991
Cholesterol (LDL)              1305880
DCSI-EGFR                       628758
DCSI-MICROALBUMIN                81297
DCSI-SERUM                     2734084
Ferritin                        219613
Height                        11808045
Hematocrit                     2574412
Hemoglobin                     2919909
Iron                            197878
NT_proBNP                         9258
PSA                             310283
Potassium                      2666981
Troponin                         94030
Weight                        14000267
eGFR                            907656
dtype: int64

In [12]:
from data_functions import  create_causal_data


## Steps in creating causal data
``` python
%%time
from data_functions import  biometrics_f, pmpm_f, canary_f, merge_filter_separate, control_outcomes, treated_outcomes, combine_treated_controls
biometrics = biometrics_f(biometrics, 'BMI', True)
pmpm = pmpm_f(pmpm, True, 12)
canary = canary_f(canary)
merged, treated, controls, canary = merge_filter_separate(biometrics, pmpm, canary, merge_how = 'inner')
controls_final = control_outcomes(controls, max_days_apart = 180, min_days_apart = 120)
treated_final, treated_help = treated_outcomes(treated, max_days_apart = 180, min_days_apart = 120)
df = combine_treated_controls(treated_final, control_finals)
```

## Create dataset

In [16]:
df, treated_help = create_causal_data(canary, pmpm, biomerics, outcome_under_study = 'BMI', drop_single = True,
                      enroll_filt = True, enroll_months = 12, merge_how = "inner",
                      max_days_apart = 180, min_days_apart = 120)

In [17]:
df.head(3)

AGE_AT_MIDMONTH  ALLOW_AMT BIOMETRIC_CATEGORY  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0               81        414                BMI        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                1          2                      0            0                0            0                  1                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0   
1               82        101                BMI        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          3                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0   
2               78         72                BMI        0             0                  0       0          0                    0             0          0              0          0           0       1                 0            1                  0                  0                     0        0                   0              0                        0                  0            1                0          7                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            1               0             0         0               0            0                   0                      0                      0           1                0              0   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CA

In [18]:
df.TREATED.value_counts()

0    112540
1       417
Name: TREATED, dtype: int64

In [19]:
Path('../../matching_data').mkdir(parents = True, exist_ok = True)
df.to_pickle('../../matching_data/data.p')
df.shape

(112957, 78)

#### SEARCH THE IMPACT OF MILESTONES_ACHIEVED ON THE OUTCOME EFFECT

In [ ]:
milestones = treated3.N_MILESTONES_ACHIEVED
milestones.value_counts().sort_index()

#### Separate Treated Groups by Milestones Achieved

In [ ]:
milestones.value_counts().sort_index()
outcomes = treated3[['OUTCOME', 'OUTCOME2', 'RESULT_VALUE']]

filt1 = milestones <= 16
filt2 = (milestones > 16)

one = outcomes[filt1].mean()
two = outcomes[filt2].mean()


#### Group 1

In [ ]:
one

#### Group 2

In [ ]:
two

In [ ]:
minimum = outcomes.OUTCOME2 == outcomes.OUTCOME2.min()

In [ ]:
treated3[minimum][['LAST_ACTIVITY_DATE','REGISTER_DATE', 'EVENT_DATE', 'OUTCOME2','MYR']]

In [ ]:
treated_help[treated_help.PERS_ID == 3153081][['LAST_ACTIVITY_DATE','REGISTER_DATE', 'EVENT_DATE', 'OUTCOME2','MYR']]

In [ ]:
treated_help[['LAST_ACTIVITY_DATE','REGISTER_DATE', 'EVENT_DATE','FF','RESULT_VALUE','OUTCOME', 'OUTCOME2','MYR']].head(2)

In [ ]:
mean0  = df[df.TREATED == 0].describe().loc[['mean', 'std']]
mean1  = df[df.TREATED == 1].describe().loc[['mean', 'std']]

means = pd.concat((mean0.T, mean1.T), axis = 1)
means.columns = ['m0', 'std0', 'm1', 'std1']

#### Compare the univariate distributions on the covariates on treatment and control groups 

In [ ]:
means.round(3)

#### Propensity score matching and distribution differences

In [ ]:
columns_to_drop = ['PERS_ID', 'BIOMETRIC_CATEGORY', 'OUTCOME', 'OUTCOME2', 'RESULT_VALUE', 'TREATED']
cols = np.setdiff1d(df.columns, columns_to_drop)
causal = df[cols]
causal[['RESULT_VALUE', 'OUTCOME', 'OUTCOME2', 'TREATED']] = df[['RESULT_VALUE', 'OUTCOME', 'OUTCOME2', 'TREATED']].copy()

In [ ]:
from sklearn.linear_model import LogisticRegression as LG
from sklearn.neighbors import NearestNeighbors

#### Balanced Logistic Regression with L2 Regularization to get the propensities

In [ ]:
model = LG(max_iter = 1000, class_weight = 'balanced').fit(causal.iloc[:,:-3].values, causal.TREATED.values)
causal['PROPENSITY'] = model.predict_proba(causal.iloc[:,:-3])[:,1]

#### 1 by 1 matching on the treated group

In [ ]:
Xtreated = causal[causal.TREATED == 1]
Xcontrols = causal[causal.TREATED == 0].reset_index(drop = True)

neighbors = NearestNeighbors(n_neighbors=1).fit(X = Xcontrols.PROPENSITY.values.reshape(-1,1))
matched_indices = neighbors.kneighbors(X = Xtreated.PROPENSITY.values.reshape(-1,1), n_neighbors=1, return_distance = False)
matched = Xcontrols.iloc[matched_indices[:,0]].reset_index(drop = True)

In [ ]:
matched.head(2)

In [ ]:
mean0m  = matched.describe().loc[['mean', 'std']]
mean1m  = Xtreated.describe().loc[['mean', 'std']]

meansm = pd.concat((mean0m.T, mean1m.T), axis = 1)
meansm.columns = ['m0', 'std0', 'm1', 'std1']

#### Treated and Matched controls distributions comparison

In [ ]:
meansm